In [1]:
from forex.pre_training_data_prep.config import config
config

{'database_name': 'django',
 'dag_id': 'NEW_prepare_forex_data',
 'list_data_columns': ['return', 'volatility', 'volume', 'lhc_mean'],
 'tz_name': 'US/Eastern',
 'price_type_name': 'mid',
 'instrument_name': 'EUR/USD',
 'interval_name': 'Minute',
 'retries_pull_forex_data': 1,
 'retry_delay_minutes_pull_forex_data': 5,
 'directory_output': '/home/emily/Desktop/projects/test/badass-data-science/badassdatascience/forecasting/deep_learning/forex/output',
 'filename_candlesticks_query_results': 'candlesticks_query_results.parquet',
 'filename_timezone_added': 'candlesticks_timezone_added.parquet',
 'filename_offset': 'candlesticks_timezone_weekday_offset.parquet',
 'filename_weekday_shift_merged': 'candlesticks_weekday_offset_merged.parquet',
 'filename_shift_days_and_hours_as_needed': 'candlesticks_shifted_as_needed.parquet',
 'filename_finalized_pandas': 'candlesticks_finalized_pandas.parquet',
 'filename_conversion_to_spark': 'spark_converted.parquet',
 'filename_pivot_and_sort': 'spark

## Get previous Pandas dataframe and put it into Spark

In [ ]:
from forex.pre_training_data_prep.tasks.spark.task_convert_pandas_df_to_spark_df import task_convert_pandas_df_to_spark_df
task_convert_pandas_df_to_spark_df(**config)

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_conversion_to_spark'])
sdf_arrays.show(5)
spark.stop()

In [2]:
from forex.pre_training_data_prep.tasks.spark.array_tasks import task_pivot_and_sort_arrays
task_pivot_and_sort_arrays(**config)

25/03/13 01:18:56 WARN Utils: Your hostname, emily-MS-7B96 resolves to a loopback address: 127.0.1.1; using 192.168.1.82 instead (on interface wlp5s0)
25/03/13 01:18:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 01:18:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from forex.pre_training_data_prep.tasks.spark.array_tasks import task_diff_the_timestamp_arrays
task_diff_the_timestamp_arrays(**config)

In [4]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_pivot_and_sort'])
sdf_arrays.show(5)
spark.stop()

+---------------+----------------------+--------------------+-----------------------+--------------------+---------------------+
|date_post_shift|sorted_timestamp_array| sorted_return_array|sorted_volatility_array| sorted_volume_array|sorted_lhc_mean_array|
+---------------+----------------------+--------------------+-----------------------+--------------------+---------------------+
|     2014-10-21|  [1413838920, 1413...|[-1.1E-4, 5.0E-5,...|   [1.3E-4, 1.9E-4, ...|[17.0, 17.0, 1.0,...| [1.28006, 1.28004...|
|     2014-10-22|  [1413925320, 1413...|[-9.0E-5, -3.0E-5...|   [9.0E-5, 3.0E-5, ...|[4.0, 5.0, 1.0, 1...| [1.27167, 1.2717,...|
|     2014-10-23|  [1414011600, 1414...|[-5.0E-5, 0.0, 0....|   [5.9E-4, 1.8E-4, ...|[37.0, 7.0, 1.0, ...| [1.2649, 1.264863...|
|     2014-10-24|  [1414098120, 1414...|[5.0E-5, 1.0E-5, ...|   [3.1E-4, 1.4E-4, ...|[17.0, 8.0, 12.0,...| [1.2648333, 1.264...|
|     2014-10-27|  [1414357200, 1414...|[-5.0E-5, -9.0E-5...|   [2.9E-4, 1.3E-4, ...|[6.0, 5.0, 1

In [5]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_timestamp_diff'])
sdf_arrays.show(5)
spark.stop()

+---------------+----------------------+--------------------+-----------------------+--------------------+---------------------+---------------------------+
|date_post_shift|sorted_timestamp_array| sorted_return_array|sorted_volatility_array| sorted_volume_array|sorted_lhc_mean_array|diff_sorted_timestamp_array|
+---------------+----------------------+--------------------+-----------------------+--------------------+---------------------+---------------------------+
|     2014-11-19|  [1416348120, 1416...|[-2.2E-4, 1.8E-4,...|   [3.4E-4, 2.2E-4, ...|[17.0, 18.0, 5.0,...| [1.2532933, 1.253...|       [1, 1, 1, 1, 1, 1...|
|     2014-11-20|  [1416434520, 1416...|[4.0E-5, 4.0E-5, ...|   [8.0E-5, 4.0E-5, ...|[4.0, 3.0, 2.0, 1...| [1.25536, 1.25538...|       [1, 1, 1, 1, 1, 1...|
|     2014-11-21|  [1416520920, 1416...|[-1.0E-4, 3.0E-5,...|   [1.0E-4, 3.0E-5, ...|[4.0, 2.0, 14.0, ...| [1.2540333, 1.254...|       [1, 1, 1, 1, 1, 1...|
|     2014-11-24|  [1416780000, 1416...|[2.0E-4, 1.2E-4, .

In [ ]:
from pyspark.sql.types import ArrayType, IntegerType, FloatType

In [ ]:
def get_all_timestamps(timestamp_array, seconds_divisor):
    return [int(x) for x in range(min(timestamp_array), max(timestamp_array) + seconds_divisor, seconds_divisor)]

udf_get_all_timestamps = f.udf(get_all_timestamps, ArrayType(IntegerType()))

##https://stackoverflow.com/questions/41190852/most-efficient-way-to-forward-fill-nan-values-in-numpy-array
#def do_nans_exist(values_array):
#    values_array = np.array([np.array(values_array)])
#    mask = np.isnan(values_array)
#    has_nan_0_or_1 = np.max([int(x) for x in mask[0]])
#    return int(has_nan_0_or_1)
#
#udf_do_nans_exist = f.udf(do_nans_exist, IntegerType())

##https://stackoverflow.com/questions/41190852/most-efficient-way-to-forward-fill-nan-values-in-numpy-array
#def do_non_nans_exist(values_array):
#    values_array = np.array([np.array(values_array)])
#    mask = ~np.isnan(values_array)
#    has_non_nan_0_or_1 = np.max([int(x) for x in mask[0]])
#    return int(has_non_nan_0_or_1)
#    
#udf_do_non_nans_exist = f.udf(do_non_nans_exist, IntegerType())


def count_nans_in_array(values_array):
    values_array = np.array([np.array(values_array)])
    mask = np.isnan(values_array)
    nan_count = np.sum([int(x) for x in mask[0]])
    return int(nan_count)
    
udf_count_nans_in_array = f.udf(count_nans_in_array, IntegerType())



##https://stackoverflow.com/questions/41190852/most-efficient-way-to-forward-fill-nan-values-in-numpy-array
#def forward_fill(values_array):
#    arr = np.array([values_array])
#    mask = np.isnan(arr)
#    idx = np.where(~mask, np.arange(mask.shape[1]), 0)
#    np.maximum.accumulate(idx, axis = 1, out = idx)
#    arr[mask] = arr[np.nonzero(mask)[0], idx[mask]]
#    to_return = list([float(x) for x in arr[0]])
#    return to_return
#
#udf_forward_fill = f.udf(forward_fill, ArrayType(FloatType()))


def locate_nans(timestamp_array, timestamp_all_array, values_array):

    # make sure we get an argsort in here later to ensure order of values is correct

    ts = np.array(timestamp_array, dtype = np.uint64) # ??
    ts_all = np.array(timestamp_all_array, dtype = np.uint64)  # we can probably make this smaller
    v = np.array(values_array, dtype = np.float64)  # we can probably make this smaller
    
    pdf = pd.DataFrame({'timestamp' : ts, 'values' : v})
    pdf_all = pd.DataFrame({'timestamp' : ts_all})

    pdf_joined = (
        pd.merge(
            pdf_all,
            pdf,
            on = 'timestamp',
            how = 'left',
        )
    )

    to_return = pdf_joined['values'].to_list()
    
    return to_return

udf_locate_nans = f.udf(locate_nans, ArrayType(FloatType()))

In [ ]:
from utilities.spark_session import get_spark_session

def meh(**config):

    spark = get_spark_session(config['spark_config'])
    sdf_arrays = (
        spark.read.parquet(config['directory_output'] + '/' + config['filename_timestamp_diff'])
        .coalesce(config['n_processors_to_coalesce'])
        .orderBy('date_post_shift')

        .drop('timestamp_argsort')   # done above just not reflected in the saved data

        .withColumn(
            'timestamps_all',
            udf_get_all_timestamps(f.col('sorted_timestamp_array'), f.lit(config['seconds_divisor']))
        )
    )

    items_list = ['return', 'volatility', 'volume', 'lhc_mean']
    
    for item in items_list:
        sdf_arrays = (
            sdf_arrays
            .withColumn(
                item + '_and_nans',
                udf_locate_nans(f.col('sorted_timestamp_array'), f.col('timestamps_all'), f.col('sorted_' + item + '_array'))
            )
            #.withColumn(
            #    item + '_forward_filled',
            #    udf_forward_fill(f.col(item + '_and_nans'))
            #)
        )

    for item in items_list:
        sdf_arrays = (
            sdf_arrays
            .withColumn(
                item + '_nan_count',
                udf_count_nans_in_array(f.col(item + '_and_nans'))
            )
        )

    # QA
    #sdf_arrays = (
    #    sdf_arrays
    #    .withColumn(
    #        'test_nan_count_equality',
    #        (
    #            (f.col('return_nan_count') == f.col('volatility_nan_count')) &
    #            (f.col('volatility_nan_count') == f.col('volume_nan_count')) &
    #            (f.col('volume_nan_count') == f.col('lhc_mean_nan_count'))
    #        )
    #    )
    #)
    #sdf_test = sdf_arrays.groupBy('test_nan_count_equality').agg(f.count(f.col('date_post_shift').alias('test_nan_count_equality_aggregated')))
    #sdf_test.show()
    #sdf_test = sdf_test.drop('test_nan_count_equality')

    sdf_arrays = sdf_arrays.withColumn('nan_count_full_day', f.col('return_nan_count'))
    for item in items_list:
        sdf_arrays = sdf_arrays.drop(item + '_nan_count')

    sdf_arrays.write.mode('overwrite').parquet(config['directory_output'] + '/' + config['filename_full_day_nans'])
    spark.stop()

meh(**config)

In [ ]:
from utilities.spark_session import get_spark_session
spark = get_spark_session(config['spark_config'])
sdf_qa = (
    spark.read.parquet(config['directory_output'] + '/' + config['filename_full_day_nans'])
    .select('nan_count_full_day', 'date_post_shift')
    .orderBy(f.col('nan_count_full_day').desc())
    .withColumn('has_nans', f.col('nan_count_full_day') > 0)
    .groupBy('has_nans').agg(f.count('date_post_shift').alias('number_of_days'))
)
sdf_qa.show()

In [ ]:
spark = get_spark_session(config['spark_config'])
sdf_qa_counts = (
    spark.read.parquet(config['directory_output'] + '/' + config['filename_full_day_nans'])
    .select('nan_count_full_day', 'date_post_shift')
    .orderBy(f.col('nan_count_full_day').desc())
)
sdf_qa_counts.show()

In [ ]:
import matplotlib.pyplot as plt
nan_counts_per_day = [x['nan_count_full_day'] for x in sdf_qa_counts.select('nan_count_full_day').collect()]

#plt.figure()
#plt.hist(nan_counts_per_day)
#plt.show()
#plt.close()

plt.figure()
plt.boxplot([nan_counts_per_day], widths=0.95)
plt.show()
plt.close()

In [ ]:
print(np.mean(nan_counts_per_day), np.median(nan_counts_per_day))

In [ ]:
pdf_qa_dates = sdf_qa_counts.toPandas()

In [ ]:
pdf_qa_dates['weekday'] = [x.weekday() for x in pdf_qa_dates['date_post_shift']]

plt.figure()
pdf_qa_dates.boxplot(by = 'weekday')
plt.show()
plt.close()

## Investigate consecutive NaNs for each day

By identifying the maximum number of NaNs in a row for a given day:

In [ ]:
from pyspark.sql.types import IntegerType, FloatType

In [ ]:
spark = get_spark_session(config['spark_config'])
sdf_arrays = spark.read.parquet(config['directory_output'] + '/' + config['filename_full_day_nans'])
sdf_arrays.show(5)

In [ ]:
#
# Had to implement a crude method to get this to work in Spark.
#
# A more elegant solution is described at:
# https://stackoverflow.com/questions/6352425/whats-the-most-pythonic-way-to-identify-consecutive-duplicates-in-a-list
#

def get_max_consecutive_NaNs(a_list):

    n_consec_nan_list = []
    count = 0
    is_in_nan_group = False
    for item in np.array(a_list):
        if np.isnan(item) or item == None:
            is_in_nan_group = True
            count += 1
        if not np.isnan(item) and is_in_nan_group:
            is_in_nan_group = False
            n_consec_nan_list.append(count)
            count = 0
        
    return max(n_consec_nan_list)
    
udf_get_max_consecutive_NaNs = f.udf(get_max_consecutive_NaNs, IntegerType())

for item in config['list_data_columns']:
    sdf_arrays = sdf_arrays.withColumn('consec_nans_' + item, udf_get_max_consecutive_NaNs(f.col(item + '_and_nans')))

sdf_arrays.select('consec_nans_return', 'consec_nans_volatility', 'consec_nans_volume', 'consec_nans_lhc_mean').show(5)

In [ ]:

#### FINISH THIS!

def multiple_args_test(*args):
    return float(np.sum(args))

udf_multiple_args_test = f.udf(multiple_args_test, FloatType())

item_list = [f.col('consec_nans_' + item) for item in config['list_data_columns']]

sdf_m_test = (
    sdf_arrays
    .withColumn('booger', udf_multiple_args_test( *item_list))
)

sdf_m_test.show(5)